At first we load the data and do the pre-processing on it 
1. check if there is any NULL in the dataset (Because it is the MNIST dataset and it is popular we did not check this part)
2. normalize the dataset with z-score and transfer the dataset to the new space with the mean of zero and the standard deviation of 1
(Normalizing is not necessary in this project because all the features are in the same range from 0 to 255 but we do not do it here because it cause a problem in the fiting of the autoencoder and it needs integers)

In [1]:
import pandas as pd
from scipy.stats import zscore

# Load the Data Set
data_labeled_train = pd.read_csv('labeled_train_set.csv')
data_labeled_test = pd.read_csv('test_set.csv')
data_unlabeled_train = pd.read_csv('unlabeled_train_set.csv')

# Conver the Data Set to Numpy Array and Spliting Labels and Features
X_train_labeled = data_labeled_train.iloc[: , 1:].values
y_train_labeled = data_labeled_train.iloc[: , 0].values
X_test_labeled = data_labeled_test.iloc[: , 1:].values
y_test_labeled = data_labeled_test.iloc[: , 0].values
X_train_unlabeled = data_unlabeled_train.iloc[:, :].values


Now we build an Auto Encoder and use it for Feature Selection and by that Reduce the dimentionality of the data
We are going to implement it in 5 layers: 
1. Input Layer
2. 3 Hidden Layers (128 , 16, 128)
3. Output Layer
So now it is obvious how the architecture of our Encoder and also Decoder is

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

# Implimenting the Auto Encoder with the architecture we talked about

# Defining the Input Image dimention
input_image = keras.Input(shape=(784,))

# Define the Encoder
encoded = layers.Dense(128, activation='relu')(input_image)
encoded = layers.Dense(16, activation='relu')(encoded)

# Build the Encoder
encoder = keras.Model(input_image, encoded)

# Define the Decoder 
encoded_input = keras.Input(shape=(16,))
decoded = layers.Dense(128, activation='relu')(encoded_input)
decoded = layers.Dense(784, activation='sigmoid')(decoded)

# Build the Decoder 
decoder = keras.Model(encoded_input, decoded)

# Build the Auto Encoder
auto_encoder = keras.Model(input_image, decoder(encoder(input_image)))

# Compile the Auto Encoder
auto_encoder.compile(optimizer="sgd", loss="mean_squared_error")

# Fit the Data on our Auto Encoder
log = auto_encoder.fit(X_train_labeled, X_train_labeled, epochs=50, batch_size=256,
shuffle=True, validation_data=(X_test_labeled, X_test_labeled))
    

Epoch 1/50
71/71 [==============================] - 1s 12ms/step - loss: 7230.7646 - val_loss: 7348.1226
Epoch 2/50
71/71 [==============================] - 1s 14ms/step - loss: 7229.5332 - val_loss: 7348.1094
Epoch 3/50
71/71 [==============================] - 1s 16ms/step - loss: 7229.5239 - val_loss: 7348.1030
Epoch 4/50
71/71 [==============================] - 1s 16ms/step - loss: 7229.5190 - val_loss: 7348.1006
Epoch 5/50
71/71 [==============================] - 1s 11ms/step - loss: 7229.5181 - val_loss: 7348.0991
Epoch 6/50
71/71 [==============================] - 1s 10ms/step - loss: 7229.5161 - val_loss: 7348.0952
Epoch 7/50
71/71 [==============================] - 1s 17ms/step - loss: 7229.5078 - val_loss: 7348.0874
Epoch 8/50
71/71 [==============================] - 1s 15ms/step - loss: 7229.4761 - val_loss: 7348.0527
Epoch 9/50
71/71 [==============================] - 1s 15ms/step - loss: 7229.4702 - val_loss: 7348.0518
Epoch 10/50
71/71 [==============================] - 1s

After Fitting of the Auto Encoder is done, we encode the data set with it (reduce its dimentionality to 16)
P.S. I decide to reduce the dimentionality of the unlabeled data and then implement clustring algorithm on it to decrease the time of the running time

In [3]:
encoded_X_train_labeled = encoder.predict(X_train_labeled)
encoded_X_test_labeled = encoder.predict(X_test_labeled)
encoded_X_train_unlabeled = encoder.predict(X_train_unlabeled)


1313/1313 [==============================] - 3s 2ms/step


Before building the MLP model, I want to cluster the unlabeled data
Clustring Algorithm : K-Means

In [4]:
from sklearn.cluster import KMeans
import numpy as np

# Build the Model
k_means = KMeans(n_clusters=10)

# Fit Model on the Unlabeled Data
k_means.fit(encoded_X_train_unlabeled)

# Get the Labels
labels = np.array(k_means.labels_)


e:\tuts\DataMining\Homeworks\HW3\AutoEncoder\env\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Post-processing 
After the clustering we have to know that which label correspond to which number from 0 to 9,
So I decide to get a sample from all of the clusters of size 10 and visually decide which label belongs to which number
Also I did not use the decoder to keep my accuracy up and because I had the previous dataset from the begining I used that and just tried to find the indexes in that dataset

In [5]:
# Get sample from all of the labels of size 10
instances_features = []
for index in range(10):
    count = 0
    for i in range(len(labels)):
        if labels[i] == index and count < 10:
            instances_features.append(X_train_unlabeled[i])
            count += 1


# Reshaping the Matrix to prepare it for the image building
print(instances_features[0])
print(len(instances_features))

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   6  82 253 253 172  44   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  10  25 191 253 252 252 252 197   6
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  30 176 252 252 253 252 252 252 252 134   0   0   

With Pillow Library, I convert the matrixes to images

In [7]:
from PIL import Image

for i in range(len(instances_features)):
    instances_features[i] = np.reshape(instances_features[i],(28, 28))
    instances_features[i] = instances_features[i].astype(np.uint8)
    img = Image.fromarray(instances_features[i], mode='L')
    img.save(f'img{i}.png')
    img.show()

KeyboardInterrupt: 

Now we Have our images and can visually recognize that which label refers to which number 
label 0 --> number 8
label 1 --> number 4
label 2 --> number 3
label 3 --> number 7
label 4 --> number 0
label 5 --> number 2
label 6 --> number 6
label 7 --> number 9
label 8 --> number 1
label 9 --> number 5

P.S. This little Sample did not give me the proper result I had to visuallize more samples from each label but for the report i just show the first 100 instances that was my first sample

Post-processing of the clustring is in the below section 

In [6]:
# Changing the labels to the numbers that belongs to that cluster
for i in range(len(labels)):
    if labels[i] == 0:
        labels[i] = 8
    elif labels[i] == 1:
        labels[i] = 4
    elif labels[i] == 2:
        labels[i] = 3
    elif labels[i] == 3:
        labels[i] = 7
    elif labels[i] == 4:
        labels[i] = 0
    elif labels[i] == 5:
        labels[i] = 2
    elif labels[i] == 6:
        labels[i] = 6
    elif labels[i] == 7:
        labels[i] = 9
    elif labels[i] == 8:
        labels[i] = 1
    elif labels[i] == 9:
        labels[i] = 5

# Combining the datasets
X_train_combined = np.concatenate((encoded_X_train_labeled, encoded_X_train_unlabeled), axis=0)
y_train_combined = np.concatenate((y_train_labeled, labels), axis=0)

Implementing the MLP Classifier 

Now we have all the data (labeled & unlabeled with our labels) and already encoded them into the 16 features space, so every thing is ready to build up the MLP Classifier

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import datetime

# Define a range of choices for each Hyper-Parameter
param_grid = {
    'hidden_layer_sizes': [(100, 100), (50, 100)],
    'activation': ['relu', 'logistic'],
    'learning_rate_init': [0.001, 0.01],
    'batch_size': [32, 64],
    'max_iter': [10, 20],}

# Create MLp Classifier
mlp = MLPClassifier(random_state=42)

# Using Five Fold Cross-Validation on our MLP Classifier
grid = GridSearchCV(mlp, param_grid=param_grid, cv=5)
t1 = datetime.datetime.now()
grid.fit(X_train_combined, y_train_combined)
t2 = datetime.datetime.now()

# Getting the results 
mlp_best = grid.best_estimator_
y_predict = mlp_best.predict(encoded_X_test_labeled)
accuracy = accuracy_score(y_test_labeled, y_predict)
precision = precision_score(y_test_labeled, y_predict, average='weighted')
recall = recall_score(y_test_labeled, y_predict, average='weighted')
f1 = f1_score(y_test_labeled, y_predict, average='weighted')
conf_mat = confusion_matrix(y_test_labeled, y_predict)
fitting_time = t2 - t1

# Print the Result
print("Accuracy : " + str(accuracy) + "\n")
print("Precision : " + str(precision) + "\n")
print("Recall : " + str(recall) + "\n")
print("F1 Score : " + str(f1) + "\n")
print("Confusion Matrix : " + str(conf_mat) + "\n")
print("Fitting Time : " + str(fitting_time) + "\n")

e:\tuts\DataMining\Homeworks\HW3\AutoEncoder\env\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
e:\tuts\DataMining\Homeworks\HW3\AutoEncoder\env\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
e:\tuts\DataMining\Homeworks\HW3\AutoEncoder\env\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
e:\tuts\DataMining\Homeworks\HW3\AutoEncoder\env\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged ye

Accuracy : 0.2358

Precision : 0.2520675798380682

Recall : 0.2358

F1 Score : 0.2326139074742693

Confusion Matrix : [[367   7  80   8  69 147 135 148  14   5]
 [  0 420  28 650   0   0  32   3   2   0]
 [ 24  33 549  29 126  25  98 133  10   5]
 [ 15 114  95 109 107  94 373  16  49  38]
 [  6 330 107  68  27  31 110  36  75 192]
 [ 36  98  86  52  14 202 326  30  31  17]
 [ 13  10 459  28   1   5 146 292   1   3]
 [  0 270  11  50   1   4  12  95 268 317]
 [ 26 120  39  18   5 245 317  20 168  16]
 [  1 308  12  35   2  24   7  13 332 275]]

Fitting Time : 0:07:30.145589



e:\tuts\DataMining\Homeworks\HW3\AutoEncoder\env\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
